In [1]:
def read_sent(path):
    ents = []
    curEnts = []
    for line in open(path):
        line = line.strip()
        if line == '':
            ents.append(curEnts)
            curEnts = []
        elif line[0] == '#' and len(line.split('\t')) == 1:
            continue
        else:
            curEnts.append(line.split('\t')[1])
    return(ents)

def read_labels(path):
    ents = []
    curEnts = []
    for line in open(path):
        line = line.strip()
        if line == '':
            ents.append(curEnts)
            curEnts = []
        elif line[0] == '#' and len(line.split('\t')) == 1:
            continue
        else:
            curEnts.append(line.split('\t')[2])
    return(ents)

def read_index(path):
    ents = []
    curEnts = []
    for line in open(path):
        line = line.strip()
        if line == '':
            ents.append(curEnts)
            curEnts = []
        elif line[0] == '#' and len(line.split('\t')) == 1:
            continue
        else:
            curEnts.append(line.split('\t')[0])
    return(ents)

In [2]:
print('test')

test


In [42]:
id2label = {0: 'O',
            1: 'B-LOC', 
            2: 'I-LOC',
            3: 'B-PER',
            4: 'B-ORG',
            5: 'I-ORG',
            6: 'I-PER'
           }

In [43]:
label2id = {label: id for id, label in id2label.items()}


In [9]:
#Training data

#returns list of lists
training_labels = read_labels("en_ewt-ud-train.iob2")
training_sent = read_sent("en_ewt-ud-train.iob2")

#flatten to one list to be able to use myutils
train_labels = sum(training_labels, [])
train_sent = sum(training_sent, [])

In [10]:
#Evaluation data

dev_labels = read_labels("en_ewt-ud-dev.iob2")
dev_sent = read_sent("en_ewt-ud-dev.iob2")

dev_flat_labels = sum(dev_labels, [])
dev_flat_sent = sum(dev_sent, [])

In [11]:
#Test data
#Keeping track of indeces to save to required .iob2 format for model's predictions

test_labels = read_labels("en_ewt-ud-test.iob2")
test_sent = read_sent("en_ewt-ud-test.iob2")
test_index = read_index("en_ewt-ud-test.iob2")

test_flat_labels = sum(test_labels, [])
test_flat_sent = sum(test_sent, [])
test_flat_index = sum(test_index, [])

In [12]:
#!pip install transformers

In [13]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [14]:
inputs = []
for sentence in training_sent:
    inputt = tokenizer(sentence, is_split_into_words=True, padding = 'max_length', max_length = 512, truncation = True)
    inputs.append(inputt)

In [15]:
inputs_dev = []
for sentence in dev_sent:
    inputt = tokenizer(sentence, is_split_into_words=True, padding = 'max_length', max_length = 512, truncation = True)
    inputs_dev.append(inputt)

In [16]:
inputs_test = []
for sentence in test_sent:
    inputt = tokenizer(sentence, is_split_into_words=True, padding = 'max_length', max_length = 512, truncation = True)
    inputs_test.append(inputt)

In [17]:
train_labels[0]


'O'

In [18]:
inputs[0].word_ids()

[None,
 0,
 1,
 2,
 3,
 4,
 5,
 5,
 5,
 6,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None

In [22]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [23]:
def list_to_sentences(lst):
    sentences = []
    current_sentence = []
    
    for item in lst:
        if item == -100:
            if current_sentence:
                sentences.append(current_sentence)
                current_sentence = []
        else:
            current_sentence.append(item)
    
    if current_sentence:
        sentences.append(current_sentence)
    
    return sentences




In [24]:
inputs[0]

{'input_ids': [101, 2777, 1107, 1103, 1362, 1110, 146, 13855, 10337, 136, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
train_labels_num[0]

In [44]:
import myutils
UNK = "[UNK]"

#id2label, label2id = myutils.labels2lookup(train_labels, UNK)

NLABELS = len(id2label)

#converting BIO labels to numerical labels
train_labels_num = [label2id.get(label, label2id) for label in train_labels]

#id2label, label2id = myutils.labels2lookup(dev_flat_labels)
#NLABELS = len(id2label)
dev_labels_num = [label2id.get(label, label2id) for label in dev_flat_labels]

#id2label, label2id = myutils.labels2lookup(test_flat_labels, UNK)
#NLABELS = len(id2label)
test_labels_num = [label2id.get(label, label2id) for label in test_flat_labels]

In [ ]:
label2id

In [ ]:
train_labels

In [45]:
train_labels_num = [label2id[label_id] for label_id in train_labels]


labels = training_labels
word_ids = []
for i in inputs:
    word_ids.append(i.word_ids())
word_ids_flat = sum(word_ids, [])
aligned_training = align_labels_with_tokens(train_labels_num, word_ids_flat)

sentences = list_to_sentences(aligned_training)

to_zip_train = [[-100] + sublist + [-100] for sublist in sentences]


# Assuming inputs and to_zip_train are defined as provided

# Zip inputs with to_zip_train and add 'label' key to each item in inputs
for input_item, label_item in zip(inputs, to_zip_train):
    input_item['labels'] = label_item

# Print the updated inputs list
print(inputs[0])

{'input_ids': [101, 2777, 1107, 1103, 1362, 1110, 146, 13855, 10337, 136, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
print(training_sent[0])

In [46]:
dev_flat_labels_num = [label2id[label_id] for label_id in dev_flat_labels]


labels = dev_labels
word_ids = []
for i in inputs_dev:
    word_ids.append(i.word_ids())
word_ids_flat = sum(word_ids, [])
aligned_dev = align_labels_with_tokens(dev_flat_labels_num, word_ids_flat)


sentences = list_to_sentences(aligned_dev)

to_zip_dev = [[-100] + sublist + [-100] for sublist in sentences]


# Assuming inputs and to_zip_train are defined as provided

# Zip inputs with to_zip_train and add 'label' key to each item in inputs
for input_item, label_item in zip(inputs_dev, to_zip_dev):
    input_item['labels'] = label_item

# Print the updated inputs list
print(inputs_dev[0])

{'input_ids': [101, 1187, 1169, 146, 1243, 182, 1766, 23694, 1116, 1107, 27629, 8223, 1161, 5952, 117, 146, 1209, 1176, 1103, 170, 14527, 17054, 2076, 117, 1133, 146, 1209, 1106, 2222, 1330, 1116, 4268, 136, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [47]:
test_flat_labels_num = [label2id[label_id] for label_id in test_flat_labels]


labels = test_labels
word_ids = []
for i in inputs_test:
    word_ids.append(i.word_ids())
word_ids_flat = sum(word_ids, [])
aligned_test = align_labels_with_tokens(test_flat_labels_num, word_ids_flat)


sentences = list_to_sentences(aligned_test)

to_zip_test = [[-100] + sublist + [-100] for sublist in sentences]


# Assuming inputs and to_zip_train are defined as provided

# Zip inputs with to_zip_train and add 'label' key to each item in inputs
for input_item, label_item in zip(inputs_test, to_zip_test):
    input_item['labels'] = label_item

# Print the updated inputs list
print(inputs_test[0])

{'input_ids': [101, 1327, 1110, 1142, 20290, 7317, 136, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [30]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [31]:
#!pip install evaluate

In [ ]:
#!pip install seqeval 
type(inputs[0][0])

In [ ]:
inputs[1]

In [32]:
import evaluate
from datasets import load_metric

metric = load_metric("seqeval")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [33]:
id2label = {v: k for k, v in label2id.items()}

In [34]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label = id2label
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
model.config.num_labels

7

In [36]:
label_names = list(label2id.keys())

In [37]:
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

In [38]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [39]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Convert labels to a list of lists if it's a set
    if isinstance(labels, set):
        labels = [labels]

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }


In [ ]:
#!pip install accelerate -U

In [40]:
from transformers import TrainingArguments

args = TrainingArguments(
    "bert-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

In [50]:
inputs_dev[0]

{'input_ids': [101, 1187, 1169, 146, 1243, 182, 1766, 23694, 1116, 1107, 27629, 8223, 1161, 5952, 117, 146, 1209, 1176, 1103, 170, 14527, 17054, 2076, 117, 1133, 146, 1209, 1106, 2222, 1330, 1116, 4268, 136, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [48]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=inputs,
    eval_dataset=inputs_dev,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics = compute_metrics
)
trainer.train()

/home/thkk/.local/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.097900,1.705763,0.006211,0.014503,0.008698,0.785553
2,0.052500,1.889014,0.002969,0.007252,0.004213,0.778071
3,0.037700,1.916486,0.003723,0.008702,0.005215,0.781955


TrainOutput(global_step=2352, training_loss=0.05918420374798937, metrics={'train_runtime': 372.7866, 'train_samples_per_second': 100.94, 'train_steps_per_second': 6.309, 'total_flos': 9832780082970624.0, 'train_loss': 0.05918420374798937, 'epoch': 3.0})

In [ ]:
model.save_pretrained('baseline')

# Evaluation

In [51]:
trainer.evaluate()

{'eval_loss': 1.9164855480194092,
 'eval_precision': 0.003723239218119764,
 'eval_recall': 0.00870195794053662,
 'eval_f1': 0.005215123859191656,
 'eval_accuracy': 0.7819553659546019,
 'eval_runtime': 6.5276,
 'eval_samples_per_second': 306.544,
 'eval_steps_per_second': 19.303,
 'epoch': 3.0}

In [ ]:
inputs_test[0]

In [56]:
predictions, labels, metrics = trainer.predict(inputs_test)
predictions = np.argmax(predictions, axis = 2)

true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    
true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

results = metric.compute(predictions = true_predictions, references = true_labels)
results

{'PER': {'precision': 0.010104199557941269,
  'recall': 0.011339475549255847,
  'f1': 0.010686258139923193,
  'number': 2822},
 'overall_precision': 0.010104199557941269,
 'overall_recall': 0.011339475549255847,
 'overall_f1': 0.010686258139923193,
 'overall_accuracy': 0.7471526195899773}

In [53]:
for i in range(len(true_predictions)):
    print("Example", i+1)
    print("Predicted:", true_predictions[i])
    print("Real:", true_labels[i])
    print()

Example 1
Predicted: ['O', 'O', 'O', 'O', 'O', 'O']
Real: ['O', 'O', 'O', 'B-PER', 'I-PER', 'O']

Example 2
Predicted: ['O', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O']
Real: ['O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O']

Example 3
Predicted: ['O', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'B-PER', 'I-PER', 'I-PER', 'I-PER', 'O']
Real: ['O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

Example 4
Predicted: ['O', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O']
Real: ['O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O']

Example 5
Predicted: ['O', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'B-PER', 'I-PER']
Real: ['O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O']

Example 6
Predicted: ['O', 'O', 'O', 'O', 'O']
Real: ['O', 'O', 'O', 'B-PER', 'O']

Example 7
Predicted: ['O', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O']
Real: ['O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'O']

Example 8
Predicted: ['O', 'O', 'O', 'O'

In [ ]:
#import pickle

# Get the trained model
#trained_model = trainer.model

# Save the trained model as a pickle file
#with open("trained_model.pkl", "wb") as f:
    #pickle.dump(trained_model, f)


In [ ]:
import pickle

with open("trained_model.pkl", "rb") as f:
    trained_model = pickle.load(f)

In [ ]:
if isinstance(trained_model, AutoModelForTokenClassification):
    print("Trained model loaded successfully!")
else:
    print("Error: Failed to load the trained model.")

In [ ]:
import pickle
from transformers import AutoModelForTokenClassification

try:
    # Open the pickle file for reading
    with open("trained_model.pkl", "rb") as f:
        # Deserialize the trained model object
        loaded_model = pickle.load(f)

    # Check if the loaded object is an instance of AutoModelForTokenClassification
    if isinstance(loaded_model, AutoModelForTokenClassification):
        print("Trained model loaded successfully!")
    else:
        print("Error: Loaded object is not an instance of AutoModelForTokenClassification.")
except Exception as e:
    print("Error occurred while loading the trained model:", e)


In [ ]:
inputs_dev[:10]

In [ ]:
inputs_test

In [ ]:
trainer.model.to('cpu')

In [ ]:
import torch

# List of sentences
sentences = test_flat_sent

# Tokenize the sentences
tokenized_inputs = tokenizer(sentences, truncation=True, padding=True, return_tensors="pt")

# Get the input tensors
input_ids = tokenized_inputs["input_ids"]
attention_mask = tokenized_inputs["attention_mask"]

# Set the model to evaluation mode
trainer.model.eval()

# Define batch size
batch_size = 16  # You can adjust this as needed


In [ ]:
print(input_ids.device.type)
print(attention_mask.device.type)

In [ ]:
import torch

# List of sentences
sentences = test_flat_sent

# Tokenize the sentences
tokenized_inputs = tokenizer(sentences, truncation=True, padding=True, return_tensors="pt")

# Get the input tensors
input_ids = tokenized_inputs["input_ids"]
attention_mask = tokenized_inputs["attention_mask"]

# Set the model to evaluation mode
trainer.model.eval()

# Define batch size
batch_size = 16  # You can adjust this as needed

# Batch processing
with torch.no_grad():
    predicted_labels = []
    for i in range(0, len(input_ids), batch_size):
        batch_input_ids = input_ids[i:i+batch_size]
        batch_attention_mask = attention_mask[i:i+batch_size]

        # Forward pass
        outputs = trainer.model(batch_input_ids, attention_mask=batch_attention_mask)

        # Get the predicted labels (class indices)
        batch_predicted_labels = torch.argmax(outputs.logits, dim=-1)

        predicted_labels.extend(batch_predicted_labels.tolist())

# Process the predictions as needed
# For example, convert logits to labels, post-process the output, etc.

# Print or use the predictions
print(predicted_labels)


In [ ]:
trainer.model